<a href="https://colab.research.google.com/github/AzevedoGabriel/AWS-MQTT-CONNECT/blob/main/C%C3%B3pia_de_Agrupamento_de_textos_com_K_Means_em_Hinos_Nacionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook foi adaptado de: https://medium.com/@lucasdesa/clusterização-de-textos-com-k-means-46254fe31bf6 e https://towardsdatascience.com/evaluating-goodness-of-clustering-for-unsupervised-learning-case-ccebcfd1d4f1

# Agrupamento de textos com K-Means

Neste notebook, usaremos o [algoritmo k-means](https://www.datascience.com/blog/k-means-clustering), um algoritmo simples e popular de __*agrupamento não-supervisionado*__, para agrupar os hinos nacionais de vários países em diferentes grupos.

O objetivo do K-means é simples: agrupar pontos de dados semelhantes e descobrir padrões subjacentes. Para atingir esse objetivo, o K-means procura um número fixo definido (k) de centróides em um conjunto de dados. Um centróide refere-se a um cluster, que é uma coleção de pontos de dados agregados devido a certas semelhanças entre si. As **médias/means** no K-means referem-se à média dos dados; isto é, encontrar o centróide. E o algoritmo é dito não supervisionado porque não temos conhecimento prévio sobre os grupos ou classes de nosso conjunto de dados, ou seja, encontraremos os grupos subjacentes em nosso conjunto de dados!

Abaixo podemos visualizar o algoritmo. Os centróides verdes correspondem aos pontos de dados mais próximos de cada um e formam clusters, então cada centróide se move para o centro de cada respectivo grupo e combina novamente os pontos de dados mais próximos entre si.

![alt text](https://github.com/lucas-de-sa/national-anthems-clustering/blob/master/Images/kmeans.gif?raw=true)

**Passos:**

__1.__ Explorar nossa coleção de hinos nacionais (corpus) <br>
__2.__ Aplicar engenharia de dados no conjunto de dados para obter o melhor desempenho do algoritmo K-means <br>
__3.__ Execute o algoritmo várias vezes, cada vez testando com um número diferente de clusters <br>
__4.__ Use diferentes métricas para visualizar nossos resultados e encontrar o melhor número de clusters (*ou seja, por que um total de X clusters é melhor do que um total de Y clusters?*) <br>
__5.__ Análise de cluster

**Métricas utilizadas para determinar o melhor número de K Cluters:**
- [Método do cotovelo](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
- [Silhouette Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

## Importando bibliotecas

In [1]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.1 MB/s eta 0:00:00


In [2]:
# Data Structures
import numpy  as np
import pandas as pd
import geopandas as gpd
import json

# Corpus Processing
import re
import nltk.corpus
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.preprocessing            import normalize

# K-Means
from sklearn import cluster

# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cmm
import seaborn            as sns
from sklearn.metrics                  import silhouette_samples, silhouette_score
from wordcloud                        import WordCloud

# Map Viz
import folium
#import branca.colormap as cm
from branca.element import Figure

import urllib.request

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Carregando o corpus

Usaremos pandas para ler o arquivo csv contendo o hino nacional de cada país e seu código de país correspondente. Os hinos foram extraídos da wikipedia e muitos deles contém palavras que usam caracteres não UTF-8 (geralmente nomes de lugares e tal), então vamos ler o arquivo com a codificação _latin1_.

Em seguida, extrairemos a coluna __Anthem__ em uma lista de textos para nosso corpus.

In [4]:
data = pd.read_csv('https://gist.githubusercontent.com/issilva5/b4a1c6dc5989ad83663ae02929f2894c/raw/28f6f957f0e9e9164027992dea52860e001ca80b/poemas.csv', encoding='utf-8')
data.columns = map(str.lower, data.columns)


data.head(6)

,author,content
0,Cecília Meireles,"Retrato\nEu não tinha este rosto de hoje,\nAss..."
1,Fernando Pessoa,"Para ser grande, sê inteiro: nada\nPara ser gr..."
2,Marina Colasanti,"Eu sei, mas não devia\nEu sei que a gente se a..."
3,Carlos Drummond de Andrade,Quadrilha\nJoão amava Teresa que amava Raimund...
4,Eugénio de Andrade,É urgente o amor\nÉ urgente o amor.\nÉ urgente...
5,Vinicius de Moraes,"Procura-se um amigo\nNão precisa ser homem, ba..."


In [7]:

def getAuthorList(data):
     authors = data['author'].to_list()
     authors = [author.lower().split(' ') for author in authors]
     authors = sum(authors, [])
     authors = list(set(authors))
     return authors

authors = getAuthorList(data)

In [8]:
corpus = data['content'].tolist()


## Processando o corpus

### 1. Stop Words and Stemming
Faremos uma rotina de engenharia de dados com nosso dataset de hinos para posteriormente fazermos um bom modelo estatístico. Para isso, removeremos todas as palavras que não contribuam para o significado semântico do texto (palavras que não estão dentro do alfabeto inglês) e manteremos todas as palavras restantes no formato mais simples possível, para que possamos aplicar uma função que dê pesos a cada palavra sem gerar nenhum viés ou outliers. Para isso existem várias técnicas para limpar nosso corpus, entre elas vamos remover as palavras mais comuns ([stop words](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/)) e aplicar [stemming](https://www.researchgate.net/figure/Stemming-process-Algorithms-of-stemming-methods-are-divided-into-three-parts-mixed_fig2_324685008), uma técnica que reduz uma palavra a é raiz.

Os métodos que aplicam a remoção de stemming e stop words estão listados abaixo. Também definiremos um método que remove todas as palavras com menos de 2 letras ou mais de 21 letras para limpar ainda mais nosso corpus.

In [ ]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

### 2. A função principal de processamento

Uma seção atrás, na exploração de nosso conjunto de dados, notamos algumas palavras contendo caracteres estranhos que deveriam ser removidos. Ao usar o RegEx, nossa principal função de processamento removerá símbolos ASCII desconhecidos, caracteres especiais, números, e-mails, URLs, etc. Ele também usa as funções auxiliares definidas acima.

In [ ]:
def processCorpus(corpus, language):
    stopwords = nltk.corpus.stopwords.words(language)
    param_stemmer = SnowballStemmer(language)
    countries_list = [line.decode('utf-8').rstrip('\n') for line in urllib.request.urlopen('https://gist.githubusercontent.com/issilva5/b4a1c6dc5989ad83663ae02929f2894c/raw/28f6f957f0e9e9164027992dea52860e001ca80b/poemas.csv')] # Load .txt file line by line
    nationalities_list = [line.decode('utf-8').rstrip('\n') for line in urllib.request.urlopen('https://gist.githubusercontent.com/issilva5/b4a1c6dc5989ad83663ae02929f2894c/raw/28f6f957f0e9e9164027992dea52860e001ca80b/poemas.csv')] # Load .txt file line by line
    other_words = [line.decode('utf-8').rstrip('\n') for line in urllib.request.urlopen('https://gist.githubusercontent.com/issilva5/b4a1c6dc5989ad83663ae02929f2894c/raw/28f6f957f0e9e9164027992dea52860e001ca80b/poemas.csv')] # Load .txt file line by line

    for document in corpus:
        index = corpus.index(document)
        corpus[index] = corpus[index].replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase

        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        listOfTokens = removeWords(listOfTokens, countries_list)
        listOfTokens = removeWords(listOfTokens, nationalities_list)
        listOfTokens = removeWords(listOfTokens, other_words)

        listOfTokens = applyStemming(listOfTokens, param_stemmer)
        listOfTokens = removeWords(listOfTokens, other_words)

        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

In [ ]:
language = 'portuguese'
corpus = processCorpus(corpus, language)


KeyboardInterrupt: ignored

### Aplicando a vetorização com TF-IDF

Agora vamos aplicar a função [TF-IDF](https://jmotif.github.io/sax-vsm_site/morea/algorithm/TFIDF.html), abreviação de frequência do documento inverso da frequência do termo, que é uma estatística numérica que se destina para refletir a importância de uma palavra para um documento em um corpus, atribuindo a cada palavra em um documento uma pontuação que varia de 0 a 1.

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

final_df = tf_idf

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)

In [ ]:
# first 5 words with highest weight on document 0:
final_df.T.nlargest(5, 0)

## Aplicando o K-Means

Função que executa o algoritmo K-Means *max_k* vezes e retorna um dicionário de cada k resultante.

In [ ]:
def run_KMeans(max_k, data):
    max_k += 1
    kmeans_results = dict()
    for k in range(2 , max_k):
        kmeans = cluster.KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 1
                               , tol = 0.0001
                               , random_state = 1
                               , algorithm = 'full')

        kmeans_results.update( {k : kmeans.fit(data)} )

    return kmeans_results

In [ ]:
# Running Kmeans
k = 8
kmeans_results = run_KMeans(k, final_df)

## Encontrando o melhor resultado

#### Método do joelho

Calcularemos o método do joelho usando a soma do quadrado das distâncias das ammostras a seus respectivos centros

In [ ]:
sum_of_squared_distances = []
n = range(2, k)
for i in n:
    sum_of_squared_distances.append(kmeans_results[i].inertia_)

plt.plot(n, sum_of_squared_distances, 'bx-')
plt.xlabel("Número de clusters")
plt.ylabel("Soma dos quadrados")
plt.title('Método do cotovelo para encontrar o k ótimo')
plt.show()

#### Silhouette Score

O Silhouette Score é uma medida de quão semelhante um objeto é ao seu próprio cluster (coesão) em comparação com outros clusters (separação).

In [ ]:
def printAvg(avg_dict):
    for avg in sorted(avg_dict.keys(), reverse=True):
        print("Avg: {}\tK:{}".format(avg.round(4), avg_dict[avg]))

def plotSilhouette(df, n_clusters, splotn, plot_idx, kmeans_labels, silhouette_avg):
    ax1 = plt.subplot(splotn, splotn, plot_idx)
    ax1.set_xlim([-0.2, 1])
    ax1.set_ylim([0, len(df) + (n_clusters + 1) * 10])

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The vertical line for average silhouette score of all the values
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.title(("K = %d, SS = %.4f" % (n_clusters, silhouette_avg)), fontsize=10, fontweight='bold')

    y_lower = 10
    sample_silhouette_values = silhouette_samples(df, kmeans_labels) # Compute the silhouette scores for each sample

    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cmm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) # Label the silhouette plots with their cluster numbers at the middle
        y_lower = y_upper + 10  # Compute the new y_lower for next plot. 10 for the 0 samples


def silhouette(kmeans_dict, df, plot=False):
    df = df.to_numpy()
    avg_dict = dict()
    subplots_num = int(np.ceil(np.sqrt(k)))
    plot_idx = 0

    if plot:
        fig = plt.figure(figsize=(15,15))

    for n_clusters, kmeans in kmeans_dict.items():
        plot_idx += 1
        kmeans_labels = kmeans.predict(df)
        silhouette_avg = silhouette_score(df, kmeans_labels) # Average Score for all Samples
        avg_dict.update( {n_clusters : silhouette_avg} )

        if(plot): plotSilhouette(df, n_clusters, subplots_num, plot_idx, kmeans_labels, silhouette_avg)

    if plot:
        plt.show()

    return avg_dict

In [ ]:
# Plotting Silhouette Analysis
ss_dict = silhouette(kmeans_results, final_df, plot=True)

In [ ]:
df_silhouette = pd.DataFrame({'n_clusters':ss_dict.keys(),'silhouette_score':ss_dict.values()})
ax = sns.lineplot(data=df_silhouette, x="n_clusters", y="silhouette_score")
ax.set(xlabel="Número de clusters", ylabel="Score", title="Média do Silhouette Score")
plt.show()

## Análise nos clusters

Agora podemos escolher o melhor número de K e dar uma olhada mais profunda em cada cluster. Olhando para os gráficos acima, temos algumas pistas de que quando K = 6 é quando os clusters são melhor definidos.

#### Histograma das palavras

Então, primeiro usaremos um histograma simples para observar as palavras mais dominantes em cada grupo:

In [ ]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = vectorizer.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Palavras mais comuns Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
best_result = 6
kmeans = kmeans_results.get(best_result)

final_df_array = final_df.to_numpy()
prediction = kmeans.predict(final_df)
n_feats = 20
dfs = get_top_features_cluster(final_df_array, prediction, n_feats)
plotWords(dfs, 13)

#### Wordcloud

Agora que podemos olhar os gráficos acima e ver as palavras mais bem pontuadas em cada cluster, também é interessante deixá-lo mais bonito fazendo um mapa de palavras de cada cluster!

In [ ]:
# Transforms a centroids dataframe into a dictionary to be used on a WordCloud.
def centroidsDict(centroids, index):
    a = centroids.T[index].sort_values(ascending = False).reset_index().values
    centroid_dict = dict()

    for i in range(0, len(a)):
        centroid_dict.update( {a[i,0] : a[i,1]} )

    return centroid_dict

def generateWordClouds(centroids):
    wordcloud = WordCloud(max_font_size=100, background_color = 'white')
    for i in range(0, len(centroids)):
        centroid_dict = centroidsDict(centroids, i)
        wordcloud.generate_from_frequencies(centroid_dict)

        plt.figure()
        plt.title('Cluster {}'.format(i))
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()

In [ ]:
centroids = pd.DataFrame(kmeans.cluster_centers_)
centroids.columns = final_df.columns
generateWordClouds(centroids)

### Visualizando no map

In [ ]:
# # Assigning the cluster labels to each country
# labels = kmeans.labels_
# data['label'] = labels
# data.head()

Agora que temos nosso agrupamento final, seria muito legal visualizá-lo em um mapa interativo. Para fazer isso, usaremos a incrível biblioteca Folium para ver nosso mapa interativo!

Carregaremos um arquivo geojson de polígonos e códigos de país com geopandas e o mesclaremos com o dataframe rotulado da célula acima.

In [ ]:
# # Map Viz
# import json
# import geopandas as gpd

# # Loading countries polygons
# geo_path = 'https://raw.githubusercontent.com/lucas-de-sa/national-anthems-clustering/master/datasets/world-countries.json'
# country_geo = json.load(urllib.request.urlopen(geo_path))
# gpf = gpd.read_file(geo_path)

# # Merging on the alpha-3 country codes
# merge = pd.merge(gpf, data, left_on='id', right_on='alpha-3')
# data_to_plot = merge[["id", "name", "label", "geometry"]]

# data_to_plot.head(3)

In [ ]:
# import branca.colormap as cm

# # Creating a discrete color map
# values = data_to_plot[['label']].to_numpy()
# color_step = cm.StepColormap(['r', 'y','g','b', 'purple', 'm'], vmin=values.min(), vmax=values.max(), caption='step')

# color_step

In [ ]:
# import folium
# from branca.element import Figure

# def make_geojson_choropleth(display, data, colors):
#     '''creates geojson choropleth map using a colormap, with tooltip for country names and groups'''
#     group_dict = data.set_index('id')['label'] # Dictionary of Countries IDs and Clusters
#     tooltip = folium.features.GeoJsonTooltip(["name", "label"], aliases=display, labels=True)
#     return folium.GeoJson(data[["id", "name","label","geometry"]],
#                           style_function = lambda feature: {
#                                'fillColor': colors(group_dict[feature['properties']['id']]),
#                                #'fillColor': test(feature),
#                                'color':'black',
#                                'weight':0.5
#                                },
#                           highlight_function = lambda x: {'weight':2, 'color':'black'},
#                           smooth_factor=2.0,
#                           tooltip = tooltip)

# # Makes map appear inline on notebook
# def display(m, width, height):
#     """Takes a folium instance and embed HTML."""
#     fig = Figure(width=width, height=height)
#     fig.add_child(m)
#     #return fig

In [ ]:
# # Initializing our Folium Map
# m = folium.Map(location=[43.5775, -10.106111], zoom_start=2.3, tiles='cartodbpositron')

# # Making a choropleth map with geojson
# geojson_choropleth = make_geojson_choropleth(["Country:", "Group:"], data_to_plot, color_step)
# geojson_choropleth.add_to(m)

# width, height = 1300, 675
# display(m, width, height)
# m

## Interpretando seus resultados

Após realizar as visualizações é importante tentar dar uma semântica a cada grupo, por exemplo, podemos ver que o grupo 1 tem um conjunto de palavras mais ligadas a religião, enquanto que o conjunto 3 tem palavras mais ligadas a liberdade.